# OCR Text recognition

https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-apis-create-account-cli?tabs=linux

https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/quickstarts-sdk/client-library?tabs=visual-studio&pivots=programming-language-python

https://docs.microsoft.com/en-us/python/api/overview/azure/cognitiveservices-vision-computervision-readme?view=azure-python

In [21]:
import os

`az login`

`az account set --subscription "<YOUR SUBSCRIPTION ID>"`

In [5]:
!az group create --name rg-aml-book-cogser --location westeurope

{
  "id": "/subscriptions/d9efb54e-0f23-46e8-bd10-0a46443e6a2d/resourceGroups/rg-aml-book-cogser",
  "location": "westeurope",
  "managedBy": null,
  "name": "rg-aml-book-cogser",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}


In [1]:
!az cognitiveservices account list-kinds -o table

Result
-----------------------------------
AnomalyDetector
CognitiveServices
ComputerVision
ContentModerator
ConversationalLanguageUnderstanding
CustomVision.Prediction
CustomVision.Training
Face
FormRecognizer
ImmersiveReader
Internal.AllInOne
LUIS
LUIS.Authoring
LanguageAuthoring
MetricsAdvisor
Personalizer
QnAMaker
QnAMaker.v2
SpeechServices
TextAnalytics
TextTranslation


If this is the first Cognitive Service you setup, then you must provision it using the Azure Portal and accept the Responsible AI terms. You will see this message:

```
This subscription cannot create ComputerVision until you agree to Responsible AI terms for this resource. You can agree to Responsible AI terms by creating a resource through the Azure Portal then trying again. For more detail go to https://go.microsoft.com/fwlink/?linkid=2164911
```

In [9]:
!az cognitiveservices account create --name computervision --resource-group rg-aml-book-cogser --kind ComputerVision --sku F0 --location westeurope --yes

{- Starting ..
  "etag": "\"5b004fd4-0000-0d00-0000-626132ed0000\"",
  "id": "/subscriptions/65073a65-465e-4673-852d-3fe1a0c28fa7/resourceGroups/rg-aml-book-cogser/providers/Microsoft.CognitiveServices/accounts/computervision",
  "identity": null,
  "kind": "ComputerVision",
  "location": "westeurope",
  "name": "computervision",
  "properties": {
    "allowedFqdnList": null,
    "apiProperties": null,
    "backupUpperLimit": 20,
    "callRateLimit": {
      "count": null,
      "renewalPeriod": null,
      "rules": [
        {
          "count": 30.0,
          "dynamicThrottlingEnabled": true,
          "key": "vision.recognizeText",
          "matchPatterns": [
            {
              "method": "POST",
              "path": "vision/recognizeText"
            },
            {
              "method": "GET",
              "path": "vision/textOperations/*"
            },
            {
              "method": "*",
              "path": "vision/read/*"
            }
          ],
     

In [46]:
region = !az cognitiveservices account show --resource-group rg-aml-book-cogser --name computervision --query location --output tsv
key = !az cognitiveservices account keys list --resource-group rg-aml-book-cogser --name computervision --query key1 --output tsv

In [61]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

import os
import time

credentials = CognitiveServicesCredentials(key)
client = ComputerVisionClient(
    endpoint="https://" + region + ".api.cognitive.microsoft.com/",
    credentials=credentials
)

# import models
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes

url = "https://github.com/Azure-Samples/cognitive-services-python-sdk-samples/raw/master/samples/vision/images/make_things_happen.jpg"
raw = True
numberOfCharsInOperationId = 36

# SDK call
rawHttpResponse = client.read(url, language="en", raw=True)

# Get ID from returned headers
operation_location = rawHttpResponse.headers["Operation-Location"]
id_location = len(operation_location) - numberOfCharsInOperationId
operation_id = operation_location[id_location:]

# Call the "GET" API and wait for it to retrieve the results 
while True:
    result = client.get_read_result(operation_id)
    if result.status not in ['notStarted', 'running']:
        break
    time.sleep(1)

# Get data
if result.status == OperationStatusCodes.succeeded:

    for line in result.analyze_result.read_results[0].lines:
        print(line.text)
        print(line.bounding_box)

make
[36.0, 132.0, 190.0, 128.0, 187.0, 191.0, 37.0, 189.0]
things
[106.0, 196.0, 274.0, 205.0, 272.0, 251.0, 103.0, 240.0]
happen
[206.0, 250.0, 411.0, 257.0, 408.0, 326.0, 204.0, 318.0]


In [64]:
!az group delete --name rg-aml-book-cogser -y

 / Finished ..